In [ ]:
import pandas as pd

def join_source_schema_by_sourceid(schema):
    sourceid = schema[schema['Path'].str.find('SourceId')>=0]
    # sourceid.to_csv('test')
    # print(sourceid)
    ValueScore = schema[schema['Path'].str.endswith('Value')]
    ValueScore = ValueScore[['ObjectId', 'Path', 'Value']]
    
    sourceid = sourceid[['ObjectId', 'Path', 'Value']]
    sourceid['extra'] = sourceid['Path'].apply(lambda st: st[0:st.find('/Sources')])
    # sourceid.to_csv('test')
    # t = t[t['Path'].str.find('SourceId') >= 0]
    ValueScore['extra'] = ValueScore['Path'].apply(lambda st: st[0:st.find('/Value')])
    # ValueScore.to_csv('test')
    # sourceid.to_csv('test')
    
    join = pd.merge(ValueScore,sourceid,how="left", on=['ObjectId','extra'])
    join = join.drop(labels=['extra','Path_y'], axis=1)
    join = join.rename(columns={'Value_y':'SourceId','Path_x':'Path','Value_x':'Value'})
    join['SourceId'] = join['SourceId'].fillna('null')
    join = join[join['SourceId'] != 'null']    
    join = join[join['Value'] != 'true']
    print(join)
    join.to_csv('test')
    # join = pd.merge(join,source,how="left", left_on='SourceId',right_on='ObjectId')

    # join = join.drop(labels=['ObjectId_y'], axis=1)
    # return join

env = pd.read_csv('/usr/yubomai/esg/schemeB/env',error_bad_lines=False)
join_source_schema_by_sourceid(env)

In [ ]:
import pandas as pd
schema = pd.read_csv('/usr/yubomai/esg/schemeB/env', error_bad_lines=False)
schema = schema[schema['Path'].str.find('AsReported')>=0 and schema['Path'].endswith('Value')]
schema = schema[schema['Value'].notna()]
schema.to_csv('reported_data')

In [ ]:
import pandas as pd

def format_schema(schema):
    print('find all source id')
    sourceid = schema[schema['Path'].str.endswith('SourceId')]
    sourceid = sourceid[sourceid['Value'].notna()]
    sourceid['extra'] = sourceid['Path'].apply(lambda st: st[0:st.find('/SourceId')])
    # sourceid.to_csv('test')
    print('find as report data')
    # print(sourceid)
    asreport = schema[schema['Path'].str.find('AsReported') >=0]
    asreport = asreport[asreport['Path'].str.endswith('Value')]
    asreport['extra'] = asreport['Path'].apply(lambda st: st[0:st.find('/AsReported[')] + '/Sources[{}]'.format(st[st.find('/AsReported[') + 12]))
    # asreport.to_csv('test')
    print('join asreport and sourceid')
    join = pd.merge(asreport,sourceid,how="left", on=['ObjectId','extra','StatementDetails/OrganizationId','StatementDetails/FinancialPeriodFiscalYear'])
    join = join.drop(labels=['extra','Path_y'], axis=1)
    join = join.rename(columns={'Value_y':'SourceId','Path_x':'Path','Value_x':'Value'})
    # join['SourceId'] = join['SourceId'].astype(str)
    return join
    # ValueScore = ValueScore[['ObjectId', 'Path', 'Value']]
    
    # sourceid = sourceid[['ObjectId', 'Path', 'Value']]
    # sourceid['extra'] = sourceid['Path'].apply(lambda st: st[0:st.find('/Sources')])
    # # sourceid.to_csv('test')
    # # t = t[t['Path'].str.find('SourceId') >= 0]
    # ValueScore['extra'] = ValueScore['Path'].apply(lambda st: st[0:st.find('/Value')])
    # # ValueScore.to_csv('test')
    # # sourceid.to_csv('test')
    
    # join = pd.merge(ValueScore,sourceid,how="left", on=['ObjectId','extra'])
    # join = join.drop(labels=['extra','Path_y'], axis=1)
    # join = join.rename(columns={'Value_y':'SourceId','Path_x':'Path','Value_x':'Value'})
    # join['SourceId'] = join['SourceId'].fillna('null')
    # join = join[join['SourceId'] != 'null']    
    # join = join[join['Value'] != 'true']
    # print(join)
    # join.to_csv('test')
    # join = pd.merge(join,source,how="left", left_on='SourceId',right_on='ObjectId')

    # join = join.drop(labels=['ObjectId_y'], axis=1)
    # return join
    
def main(source_path, schema_path):
    # read source
    source = pd.read_csv(source_path,error_bad_lines=False)
    SourceAbstract = source[source['Path'] == 'Sources/SourceAbstract'][['ObjectId','Value']]
    SourcePublisher= source[source['Path'] == 'Sources/SourcePublisher'][['ObjectId','Value']]
    SourceUrl= source[source['Path'] == 'Sources/SourceUrl'][['ObjectId','Value']]
    SourceTitle = source[source['Path'] == 'Sources/SourceTitle'][['ObjectId','Value']]
    SourceAbstract = pd.merge(SourceAbstract,SourcePublisher,how="left", on="ObjectId")
    SourceAbstract = pd.merge(SourceAbstract,SourceUrl,how="left", on="ObjectId")
    SourceAbstract = pd.merge(SourceAbstract,SourceTitle,how="left", on="ObjectId")
    SourceAbstract.columns = ['ObjectId', 'Abstract', 'Publisher', 'Url', 'Title']
    SourceAbstract['ObjectId'] = SourceAbstract['ObjectId'].astype(str)
    # SourcePublisher.to_csv('test')

    print('handle source done')
    # read scheme
    scheme = pd.read_csv(schema_path,error_bad_lines=False, keep_default_na=False)
    scheme = format_schema(scheme)
    scheme['SourceId'] = scheme['SourceId'].astype(str)
    # print(SourcePublisher.info())
    # print(scheme.info())
    # scheme.to_csv('test')
    print('handle schema done')
    # join source and schema
    res = pd.merge(scheme,SourceAbstract,how="left", left_on='SourceId',right_on='ObjectId')
    print('join source and schema done')

    res = res[res['Url'].notna()]
    res.to_csv('/data/env', index=False)
    print('dump res done')
    


main('/usr/yubomai/esg/sources/all.csv', '/usr/yubomai/esg/schemeB/env')

# env = pd.read_csv('/usr/yubomai/esg/schemeB/env',error_bad_lines=False,nrows=100000)
# join_source_schema_by_sourceid(env)

In [ ]:
import pandas as pd
res = pd.read_csv('/home/linzhisheng/esg/res',error_bad_lines=False)
res = res[res['Url'].notna()]
res.to_csv('env.csv')

In [3]:
# filter table data
import pandas as pd
from decimal import Decimal

def is_chinese(string):
    for ch in string:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()
# def check_data(x):
#     print(x[0])
source = pd.read_csv('data/all')
print(len(source))
count = 0
res = pd.DataFrame(columns=['value','text','type','path','url'])
for value,text,type,url in zip(source['Value'],source['Abstract'], source['Path'],source['Url']):
    path = type[type.find('/') + 1:type.find('/AsReported[')]
    type = type[0:type.find('DataPoints')]
    # print(type)
    if int(value) == value:
        value = int(value)
    format_value = format(value,',')
    # if text.find('\t') < 0 and is_chinese(text) == False:
    #     count = count + 1
    if text.find(str(format_value)) >=0:
        # res.loc[count] = [format_value,text,type]
        
        left = text.find(str(format_value))
        right = text.find(str(format_value)) + len(str(format_value))
        if left - 2 >=0 and text[left-2] !=' ' and right + 2 < len(text) and text[right+2] !=' ':
            if not text[left-2].isdigit() and not text[right+2].isdigit():
                res.loc[count] = [format_value,text,type,path,url]
                count = count + 1
        
        
    elif text.find(str(value)) >=0:
        left = text.find(str(value))
        right = text.find(str(value)) + len(str(value))
        if left - 2 >=0 and text[left-2] !=' ' and right + 2 < len(text) and text[right+2] !=' ':
            if not text[left-2].isdigit() and not text[right+2].isdigit():
                res.loc[count] = [str(value),text,type,path,url]
                count = count + 1
    if count !=0 and count % 10000 ==0:
        print(count)
        # break
print(count)
print(count/len(source))
res.to_csv('data/train_fine_grained_url.csv', index=False)


237410
10000
20000
20000
20000
20000
20000
20000
30000
30000
30000
30000
30000
30000
30000
30000
30000
30000
30000
30000
31509
0.13271976749083864


In [ ]:
import pandas as pd
env = pd.read_csv('env')
gov = pd.read_csv('gov')
social = pd.read_csv('social')
env = pd.concat([env,gov])
env = pd.concat([env,social])
env.to_csv('all')

In [ ]:
t = 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'
t.find('Saint Bernadette Soubirous')

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset('csv', data_files='data/train.csv')
raw_datasets = raw_datasets['train'].train_test_split(test_size=0.1,seed=42)
# print(raw_datasets['train'][0])
raw_datasets['test'].to_csv('test.csv', index=False)
# print(raw_datasets['test'][0])

In [ ]:
from transformers import AutoTokenizer
checkpoint = 'bert-finetuned-esgQA-fine-grained-8-2'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# text = '49, 100. 4'
text = '49,100.4'
input = tokenizer(text)
print(input['input_ids'])
print(tokenizer.decode(input['input_ids'], skip_special_tokens=True))




In [ ]:
from datasets import *
import re
import json
def question_set(example, template):
    answer_set = example['answer_set']
    id = 0
    format_answer_set = {
        'question':[],
        'context':[],
        'id':[]
    }
    for answer in answer_set:
        if re.search(r'\d', answer['answer']):
            id = id + 1
            format_answer_set['id'].append(str(id))
            format_answer_set['question'].append(template.format(answer['answer']))
            format_answer_set['context'].append(answer['context'].split('[SEP]')[1])

    return Dataset.from_dict(format_answer_set)

source = load_dataset('csv', data_files='data/figure.csv')
print(source['train']['answer_set'][0])
source = json.loads(source['train']['answer_set'][0])

# print(question_set(source['train'],'What is {} about?'))

In [4]:
import requests
response = requests.get('http://www.onsemi.com/pub/Collateral/BRD8084-D.PDF')
print(response.status_code)
# with open('metadata.pdf', 'wb') as f:
#     f.write(response.content)

200
